In [ ]:
%flink_sql_replace -s '{{{{.Values.flink.ngsildUpdateWindow|squote}}}}' -r "'0.001'"

In [ ]:
%%flink_gateway_sql_prepare
SET 'pipeline.name' = 'NGSI-LD insert job';

In [ ]:
%%flink_gateway_sql_query
            /* aggregates entities from attributes topic and insert it every 2 seconds in update channel
            part of CORE SERVICES */
            insert into ngsild_updates
                select 'update', true, true, '[{"id": "' || id || '",' || LISTAGG(attributeagg) || '}]'
                from (
                    select  window_start, window_end, B.entityId as id, '"' || B.name || '":[' || LISTAGG(B.indexagg) || ']' as attributeagg
                    from(
                        select window_start, window_end, entityId, name, `datasetId`,
                        '{"observedAt": "' || LAST_VALUE(DATE_FORMAT(A.`ts`,  'yyyy-MM-dd''T''HH:mm:ss.') || CAST(EXTRACT(MILLISECOND from A.`ts`) as STRING)) || 'Z", "type": "'
                                || LAST_VALUE(A.`type`)
                                || '", "datasetId": "'
                                || IF(A.`datasetId` IS NOT NULL, A.`datasetId`, '@none')
                                || IF(LAST_VALUE(A.`type`) =  'https://uri.etsi.org/ngsi-ld/Relationship','", "object": "', '", "value": ')
                                || IF(LAST_VALUE(A.`type`) =  'https://uri.etsi.org/ngsi-ld/Relationship', LAST_VALUE(A.`attributeValue`) || '"',
                                        
                                        IF((LAST_VALUE(A.nodeType) <> '@json' AND LAST_VALUE(A.nodeType) <> '@id') OR LAST_VALUE(A.nodeType) IS NULL, '"' || LAST_VALUE(A.`attributeValue`) || '"',
                                            /* @id needs encoding with "@id": value */
                                            IF(LAST_VALUE(A.nodeType) = '@id', '{"@id": "' || LAST_VALUE(A.`attributeValue`) || '"}',
                                            /* @json is put directly into value field */
                                                IF(LAST_VALUE(A.nodeType) = '@json', LAST_VALUE(A.`attributeValue`), 'null' /*should not happen */)))
                                )
                                || '}' as indexagg
                        FROM TABLE( TUMBLE(TABLE attributes, DESCRIPTOR(ts), INTERVAL {{.Values.flink.ngsildUpdateWindow|squote}} second)) as A
                        WHERE A.entityId IS NOT NULL and (A.deleted IS NULL or A.deleted = false) and (A.synced IS NULL or A.synced = false)
                        GROUP BY A.`id`, A.`datasetId`, A.`entityId`, A.`name`, window_start, window_end) as B
                    GROUP BY window_start, window_end, `entityId`, `name`, `datasetId`
                ) GROUP BY window_start, window_end, id;

In [ ]:
%%flink_gateway_sql_prepare
SET 'pipeline.name' = 'Attribute insert job';

In [ ]:
%flink_sql_replace -s '{{{{ .Values.flink.attributeInsertWindow |squote }}}}' -r "'0.001'"

In [ ]:
%%flink_gateway_sql_query
           /* forward only non-retracted attribute changes
            This is part of the core service */
            insert into attributes_writeback
            select id, parentId, entityId, name, nodeType, valueType, `type`, `attributeValue`, `datasetId`, `unitCode`, `deleted`, `synced` from
            (select id, last_value(parentId) as parentId, last_value(entityId) as entityId, last_value(name) as name, last_value(nodeType) as nodeType,
                last_value(valueType) as valueType , last_value(`type`) as `type`,
            last_value(`attributeValue`) as `attributeValue`,
            `datasetId` as `datasetId`, last_value(`unitCode`) as `unitCode`, last_value(`deleted`) as `deleted`, last_value(`synced`) as `synced`,
              TUMBLE_START(ts, INTERVAL {{ .Values.flink.attributeInsertWindow |squote }} SECOND),
              TUMBLE_END(ts, INTERVAL {{ .Values.flink.attributeInsertWindow |squote }} SECOND)
            from attributes_insert
            group by `id`, `datasetId`, TUMBLE(ts, INTERVAL {{ .Values.flink.attributeInsertWindow |squote }} SECOND));

In [ ]:
%%flink_gateway_sql_prepare
DROP TABLE IF EXISTS `alerts`;
CREATE TABLE `alerts` (
    `resource` STRING,
    `event` STRING,
    `environment` STRING,
    `service` ARRAY < STRING >,
    `severity` STRING,
    `customer` STRING,
    `text` STRING,
    PRIMARY KEY (resource, event) NOT ENFORCED
) WITH (
    'connector' = 'upsert-kafka',
    'value.format' = 'json',
    'value.json.fail-on-missing-field' = 'False',
    'value.json.ignore-parse-errors' = 'True',
    'key.format' = 'json',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'topic' = 'iff.alerts'
);
DROP TABLE IF EXISTS `alerts`;
CREATE TABLE `alerts` (
    `resource` STRING,
    `event` STRING,
    `environment` STRING,
    `service` ARRAY < STRING >,
    `severity` STRING,
    `customer` STRING,
    `text` STRING
) WITH (
    'connector' = 'kafka',
    'format' = 'json',
    'json.fail-on-missing-field' = 'False',
    'json.ignore-parse-errors' = 'True',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'scan.startup.mode' = 'latest-offset',
    'topic' = 'iff.alerts'
);
DROP TABLE IF EXISTS `alerts_bulk`;
CREATE TABLE `alerts_bulk` (
    `resource` STRING,
    `event` STRING,
    `environment` STRING,
    `service` ARRAY < STRING >,
    `severity` STRING,
    `customer` STRING,
    `text` STRING,
    watermark FOR ts AS ts - INTERVAL '0.001' SECONDS,
    `ts` TIMESTAMP(3) METADATA
    FROM
        'timestamp' VIRTUAL,
        PRIMARY KEY (resource, event) NOT ENFORCED
) WITH (
    'connector' = 'upsert-kafka',
    'value.format' = 'json',
    'value.json.fail-on-missing-field' = 'False',
    'value.json.ignore-parse-errors' = 'True',
    'key.format' = 'json',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'topic' = 'iff.alerts.bulk'
);
DROP TABLE IF EXISTS `attributes`;
 CREATE TABLE `attributes` (
    `id` STRING,
    `entityId` STRING,
    `name` STRING,
    `nodeType` STRING,
    `valueType` STRING,
    `index` INTEGER,
    `type` STRING,
    `https://uri.etsi.org/ngsi-ld/datasetId` STRING,
    `https://uri.etsi.org/ngsi-ld/hasValue` STRING,
    `https://uri.etsi.org/ngsi-ld/hasObject` STRING,
    `ts` TIMESTAMP(3) METADATA
    FROM
        'timestamp' VIRTUAL,
        watermark FOR ts AS ts
) WITH (
    'connector' = 'kafka',
    'format' = 'json',
    'json.fail-on-missing-field' = 'False',
    'json.ignore-parse-errors' = 'True',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'scan.startup.mode' = 'latest-offset',
    'topic' = 'iff.ngsild.attributes'
);
DROP TABLE IF EXISTS `attributes_writeback`;
 CREATE TABLE `attributes_writeback` (
    `id` STRING,
    `entityId` STRING,
    `name` STRING,
    `nodeType` STRING,
    `valueType` STRING,
    `index` INTEGER,
    `type` STRING,
    `https://uri.etsi.org/ngsi-ld/datasetId` STRING,
    `https://uri.etsi.org/ngsi-ld/hasValue` STRING,
    `https://uri.etsi.org/ngsi-ld/hasObject` STRING,
    PRIMARY KEY (id, index) NOT ENFORCED
) WITH (
    'connector' = 'upsert-kafka',
    'value.format' = 'json',
    'value.json.fail-on-missing-field' = 'False',
    'value.json.ignore-parse-errors' = 'True',
    'key.format' = 'json',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'topic' = 'iff.ngsild.attributes'
);
DROP TABLE IF EXISTS `attributes_insert`;
 CREATE TABLE `attributes_insert` (
    `id` STRING,
    `entityId` STRING,
    `name` STRING,
    `nodeType` STRING,
    `valueType` STRING,
    `index` INTEGER,
    `type` STRING,
    `https://uri.etsi.org/ngsi-ld/datasetId` STRING,
    `https://uri.etsi.org/ngsi-ld/hasValue` STRING,
    `https://uri.etsi.org/ngsi-ld/hasObject` STRING,
    `ts` TIMESTAMP(3) METADATA
    FROM
        'timestamp' VIRTUAL,
        watermark FOR `ts` AS `ts`,
        PRIMARY KEY (id, index) NOT ENFORCED
) WITH (
    'connector' = 'upsert-kafka',
    'value.format' = 'json',
    'value.json.fail-on-missing-field' = 'False',
    'value.json.ignore-parse-errors' = 'True',
    'key.format' = 'json',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'topic' = 'iff.ngsild.attributes_insert'
);
DROP TABLE IF EXISTS `attributes_insert_filter`;
 CREATE TABLE `attributes_insert_filter` (
    `id` STRING,
    `entityId` STRING,
    `name` STRING,
    `nodeType` STRING,
    `valueType` STRING,
    `index` INTEGER,
    `type` STRING,
    `https://uri.etsi.org/ngsi-ld/datasetId` STRING,
    `https://uri.etsi.org/ngsi-ld/hasValue` STRING,
    `https://uri.etsi.org/ngsi-ld/hasObject` STRING,
    `ts` TIMESTAMP(3) METADATA
    FROM
        'timestamp',
        watermark FOR `ts` AS `ts`
) WITH (
    'connector' = 'kafka',
    'format' = 'json',
    'json.fail-on-missing-field' = 'False',
    'json.ignore-parse-errors' = 'True',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'scan.startup.mode' = 'latest-offset',
    'topic' = 'iff.ngsild.attributes_insert'
);
DROP TABLE IF EXISTS `ngsild_updates`;
 CREATE TABLE `ngsild_updates` (
    `op` STRING,
    `overwriteOrReplace` Boolean,
    `noForward` Boolean,
    `entities` STRING
) WITH (
    'connector' = 'kafka',
    'format' = 'json',
    'json.fail-on-missing-field' = 'False',
    'json.ignore-parse-errors' = 'True',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'scan.startup.mode' = 'latest-offset',
    'topic' = 'iff.ngsild-updates'
);
DROP VIEW IF EXISTS `attributes_insert_view`;
CREATE VIEW `attributes_insert_view` AS
SELECT
    id,
    entityId,
    name,
    nodeType,
    valueType,
    index,
    `type`,
     'https://uri.etsi.org/ngsi-ld/datasetId',
     `https://uri.etsi.org/ngsi-ld/hasValue`,
     `https://uri.etsi.org/ngsi-ld/hasObject`,
     `ts` FROM (
        SELECT
            *,
             ROW_NUMBER() OVER (
                PARTITION BY `id`,
                `index` 
                ORDER BY
                    ts DESC
            ) AS rownum 
        FROM
            `attributes_insert_filter`
    ) 
WHERE
    rownum = 1
    and entityId is NOT NULL;

In [ ]:
%flink_sql_replace -s '{{{{.Values.flink.alertWindow | squote}}}}' -r "'0.001'"

In [ ]:
%%flink_gateway_sql_query
    
WITH WindowedAlerts AS (
    SELECT 
        `resource`, 
        `event`,
        `environment`,
        `service`,
        `customer`,
        LAST_VALUE(severity) AS `severity`,
        LAST_VALUE(`text`) AS `text`,
        TUMBLE_START(`ts`, INTERVAL '0.001' SECOND) AS window_start,
        TUMBLE_END(`ts`, INTERVAL '0.001' SECOND) AS window_end,
        TUMBLE_ROWTIME(`ts`, INTERVAL '0.001' SECOND) AS window_time
    FROM TABLE(
        TUMBLE(TABLE alerts_bulk, DESCRIPTOR(ts), INTERVAL '0.001' SECOND)
    )
    GROUP BY 
        `resource`, 
        `event`, 
        `service`, 
        `customer`, 
        `environment`, 
        TUMBLE(`ts`, INTERVAL '0.001' SECOND)
),
AlertsWithPreviousSeverity AS (
    SELECT
        `resource`,
        `event`,
        `environment`,
        `service`,
        `customer`,
        `severity`,
        `text`,
        `window_end`,
        `window_time`,
        LAG(`severity`) OVER (
            PARTITION BY `resource`, `event`, `service`, `customer`, `environment`
            ORDER BY `window_time`
        ) AS prev_severity
    FROM WindowedAlerts
)
SELECT 
    `resource`,
    `event`,
    `environment`,
    `service`,
    `severity`,
    `customer`,
    `text`
FROM 
    AlertsWithPreviousSeverity
WHERE 
    prev_severity IS NULL OR severity <> prev_severity;




In [ ]:
%%flink_gateway_sql_query
select window_start, window_end, entityId, name, `datasetId`,
                        '{"observedAt": "' || LAST_VALUE(DATE_FORMAT(A.`ts`,  'yyyy-MM-dd''T''HH:mm:ss.') || CAST(EXTRACT(MILLISECOND from A.`ts`) as STRING)) || 'Z", "type": "'
                                || LAST_VALUE(A.`type`)
                                || '", "datasetId": "'
                                || IF(A.`datasetId` IS NOT NULL, A.`datasetId`, '@none')
                                || IF(LAST_VALUE(A.`type`) =  'https://uri.etsi.org/ngsi-ld/Relationship','", "object": "', '", "value": ')
                                || IF(LAST_VALUE(A.`type`) =  'https://uri.etsi.org/ngsi-ld/Relationship', LAST_VALUE(A.`attributeValue`) || '"',
                                        
                                        IF((LAST_VALUE(A.nodeType) <> '@json' AND LAST_VALUE(A.nodeType) <> '@id') OR LAST_VALUE(A.nodeType) IS NULL, '"' || LAST_VALUE(A.`attributeValue`) || '"',
                                            /* @id needs encoding with "@id": value */
                                            IF(LAST_VALUE(A.nodeType) = '@id', '{"@id": "' || LAST_VALUE(A.`attributeValue`) || '"}',
                                            /* @json is put directly into value field */
                                                IF(LAST_VALUE(A.nodeType) = '@json', LAST_VALUE(A.`attributeValue`), 'null' /*should not happen */)))
                                )
                                || '}' as indexagg
                        FROM TABLE( TUMBLE(TABLE attributes, DESCRIPTOR(ts), INTERVAL '0.001' second)) as A
                        WHERE A.entityId IS NOT NULL and (A.deleted IS NULL or A.deleted = false) and (A.synced IS NULL or A.synced = false)
                        GROUP BY A.`id`, A.`datasetId`, A.`entityId`, A.`name`, window_start, window_end;